In [3]:
import numpy as np
import pandas as pd 

import data_api
import helpers

## 1. Loading the datasets

In [9]:
# Create the data_api client and set the number of records to be returned
client = data_api.Client()
num_records = 100

# Note: since we are not providing an api token to the client, it is best to keep the number 
# of records returned low

In [5]:
df_311 = client.load_311(limit=num_records)
df_complaint_problems = client.load_complaint_problems(limit=num_records)
df_dob_omplaints = client.load_dob_complaints(limit=num_records)

In [6]:
df_311.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,longitude,location,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,closed_date,resolution_description,resolution_action_updated_date
0,50939912,2021-06-20T02:08:04.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11211,186 NORTH 6 STREET,NORTH 6 STREET,...,-73.9575630010249,"{'latitude': '40.71680296321391', 'longitude':...",17613,36,2,38,57,NaN,NaN,NaN
1,50932349,2021-06-20T02:07:52.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10456,735 EAST 163 STREET,EAST 163 STREET,...,-73.90319411682076,"{'latitude': '40.82271708791926', 'longitude':...",10934,34,5,42,25,NaN,NaN,NaN
2,50932392,2021-06-20T02:07:52.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,11233,173 THOMAS S BOYLAND STREET,THOMAS S BOYLAND STREET,...,-73.91361695887984,"{'latitude': '40.67872765229688', 'longitude':...",13516,55,2,17,46,NaN,NaN,NaN
3,50936403,2021-06-20T02:07:34.000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10452,40 MARCY PLACE,MARCY PLACE,...,-73.91767022044058,"{'latitude': '40.83861748382105', 'longitude':...",10930,50,5,42,27,NaN,NaN,NaN
4,50939185,2021-06-20T02:07:28.000,NYPD,New York City Police Department,Noise - Commercial,Car/Truck Music,Store/Commercial,10452,1552 JEROME AVENUE,JEROME AVENUE,...,-73.91523885071975,"{'latitude': '40.84363854126145', 'longitude':...",10930,50,5,29,27,NaN,NaN,NaN


In [7]:
df_complaint_problems.head()

,problemid,complaintid,unittypeid,unittype,spacetypeid,spacetype,typeid,type,majorcategoryid,majorcategory,minorcategoryid,minorcategory,codeid,code,statusid,status,statusdate,statusdescription
0,17307278,8412850,91,APARTMENT,543,ENTIRE APARTMENT,1,EMERGENCY,56,DOOR/WINDOW,337,WINDOW FRAME,2836,LOOSE OR DEFECTIVE,2,CLOSE,2017-03-31T00:00:00.000,The Department of Housing Preservation and Dev...
1,17317058,8417365,91,APARTMENT,543,ENTIRE APARTMENT,3,NON EMERGENCY,63,UNSANITARY CONDITION,376,PESTS,2821,MICE,2,CLOSE,2017-03-16T00:00:00.000,The Department of Housing Preservation and Dev...
2,17016467,8249017,91,APARTMENT,545,ENTRANCE/FOYER,1,EMERGENCY,56,DOOR/WINDOW,333,DOOR,2665,LOCK BROKEN OR MISSING,2,CLOSE,2017-03-03T00:00:00.000,The Department of Housing Preservation and Dev...
3,14548958,6967900,91,APARTMENT,541,BATHROOM,1,EMERGENCY,9,PLUMBING,63,BATHTUB/SHOWER,2538,BROKEN OR MISSING,2,CLOSE,2014-07-29T00:00:00.000,The Department of Housing Preservation and Dev...
4,14548959,6967900,91,APARTMENT,541,BATHROOM,3,NON EMERGENCY,9,PLUMBING,63,BATHTUB/SHOWER,2540,FAUCET BROKEN/MISSING/LEAKING,2,CLOSE,2014-08-04T00:00:00.000,The Department of Housing Preservation and Dev...


In [8]:
df_dob_omplaints.head()

,complaint_number,status,date_entered,house_number,zip_code,house_street,bin,community_board,special_district,complaint_category,unit,disposition_date,disposition_code,inspection_date,dobrundate
0,1000006,CLOSED,12/30/1988,101,10003,1 AVENUE,1006289,103,,02,MAN.,01/04/1989,I1,01/03/1989,2018-11-07T00:00:00.000
1,1000008,CLOSED,01/03/1989,101,10003,1 AVENUE,1006289,103,,03,MAN.,01/04/1989,I2,01/03/1989,2018-11-07T00:00:00.000
2,1000009,CLOSED,01/03/1989,101,10003,1 AVENUE,1006289,103,,01,MAN.,01/04/1989,I2,01/04/1989,2018-11-07T00:00:00.000
3,1000010,CLOSED,01/03/1989,333,10013,HUDSON STREET,1010374,102,,67,C & D,01/06/1989,I2,01/04/1989,2018-11-07T00:00:00.000
4,1000011,CLOSED,01/03/1989,333,10013,HUDSON STREET,1010374,102,,10,MAN.,01/09/1989,H1,01/03/1989,2018-11-07T00:00:00.000


## 2. Extracting useful columns